# Prompts
- 入力プロンプトをいい感じに扱える

## PromptTemplate
- プロンプトをテンプレート化できる

In [1]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを教えてください。
できるだけ簡単に作れるレシピでお願いします。
また出力はマークダウン形式でお願いします。

料理名: {dish}
"""

prompt = PromptTemplate(
    input_variables=['dish'],
    template=template,
)

result = prompt.format(dish='カレー')
print(result)


以下の料理のレシピを教えてください。
できるだけ簡単に作れるレシピでお願いします。
また出力はマークダウン形式でお願いします。

料理名: カレー



In [2]:
# 文字列型でも同じことができるので、この使い方だと旨味は無さそう
result = template.format(dish='カレー')
print(result)


以下の料理のレシピを教えてください。
できるだけ簡単に作れるレシピでお願いします。
また出力はマークダウン形式でお願いします。

料理名: カレー



## ChatPromptTemplate
- Chat Completions API形式（チャット形式）に対応させたもの
- システム(System)、ユーザー(Human)、アシスタント(AI) それぞれをテンプレート化してまとめて扱える

In [15]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,  # システム
    HumanMessagePromptTemplate,  # ユーザー
)

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template('あなたは{country}料理のプロフェッショナルです。'),
    HumanMessagePromptTemplate.from_template('以下の料理のレシピを教えてください\n\n料理名: {dish}')
])

messages = chat_prompt.format_prompt(country='イギリス', dish='肉じゃが').to_messages()
messages

[SystemMessage(content='あなたはイギリス料理のプロフェッショナルです。'),
 HumanMessage(content='以下の料理のレシピを教えてください\n\n料理名: 肉じゃが')]

In [18]:
# アシスタント（AI）も追加した版
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,  # システム
    HumanMessagePromptTemplate,  # ユーザー
    AIMessagePromptTemplate,  # アシスタント
)

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template('あなたは{country}料理のプロフェッショナルです。'),
    HumanMessagePromptTemplate.from_template('ユーザーに料理名を質問し、その料理のレシピを教えてください。'),
    AIMessagePromptTemplate.from_template('レシピを知りたい料理名を教えてください'),
    HumanMessagePromptTemplate.from_template('{country}風{dish}'),
])

messages = chat_prompt.format_prompt(country='イギリス', dish='肉じゃが').to_messages()
messages

[SystemMessage(content='あなたはイギリス料理のプロフェッショナルです。'),
 HumanMessage(content='ユーザーに料理名を質問し、その料理のレシピを教えてください。'),
 AIMessage(content='レシピを知りたい料理名を教えてください'),
 HumanMessage(content='イギリス風肉じゃが')]

In [19]:
# ChatGPTに投げてみる
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

result = chat(messages)
print(result.content)

イギリス風肉じゃがのレシピをお伝えしますね。

【材料】
- 牛肉（シチュー用）：500g
- じゃがいも：4個（約600g）
- 玉ねぎ：1個
- 人参：1本
- セロリ：1本
- トマト缶：1缶（400g）
- ビーフブイヨン：500ml
- 小麦粉：大さじ2
- オリーブオイル：大さじ2
- 塩、こしょう：適量

【作り方】
1. 牛肉を食べやすい大きさに切り、塩とこしょうで下味をつける。
2. じゃがいも、玉ねぎ、人参、セロリをそれぞれ食べやすい大きさに切る。
3. 大きめの鍋にオリーブオイルを熱し、牛肉を焼き色がつくまで炒める。
4. 牛肉に小麦粉をまぶし、さらに炒める。
5. 野菜を加えて炒め、トマト缶とビーフブイヨンを加える。
6. 沸騰したら火を弱め、蓋をして約1時間半から2時間煮込む。途中、アクを取り除く。
7. じゃがいもを加え、さらに30分ほど煮込む。
8. 味を調整し、完成です。

ぜひお試しください。


## Example selectors
- Few-shotプロんプティングで使用する例（質問＆回答例）を選択する機能
- 動的に例を選択し埋め込める
- 固定の例を埋め込む場合は使わなくていい
- 詳細は割愛